In [21]:
# "Проверка гипотезы: Разворот зависит от предыдущих курсов (вернее, разниц с текущей) и, возможно, volumes. 
# Задача  разбивается на следующие: 1) определение разворота 2) тестированию гипотезы 
# Пока я в процессе решеения первой задачи.    

In [22]:
from get_data import get_polo_data
import math
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn import metrics 
from sklearn import svm
from pandas import DataFrame
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import plotly.graph_objects as go
import numpy as np

In [23]:
# РАБОТАЮ с пятиминутками! 
sd = get_polo_data(pair="USDT_BTC", frame=(4)*60*60*24,zoom='5MIN')
print("Download from polo",len(sd))
print("Errors values",len(sd)-len(sd.dropna()))

DEBUG:poloniex:Getting 2019-12-10 20:35:00 - 2019-12-10 20:39:22 USDT_BTC candles from Poloniex...
DEBUG:poloniex:Updating USDT_BTC database with 1 entrys...
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 250.00it/s]
DEBUG:poloniex:Getting USDT_BTC chart data from db


Download from polo 1152
Errors values 0


In [31]:
# Решил сделать в виде функций, чтобы отлаживать в одной строчке меняя параметры (ниже) и было видно график

def SetDataFrameFromLagPrec(lags, prec):  # Функция "рассчитывает" развороты. Сделано топонрно, про оптимизацию нее говорим. 
    for i in range(0, lags): #создаем столбцы, в каждом - разница между текущим и i (макс количество передается в функции)
        sd["Lag%s" % str(i+1)] = sd["open"]-sd["open"].shift(i+1)
    for z, row in sd.iterrows():    #пробегаем по таблице и в столбец direction пишем куда движется тренд. 
        n_neg,n_pos=0,0             #1 - вверх, 0 - хрен знает, -1  вниз 
        for i in range(0, lags):
            if row["Lag%s" % str(i+1)] < 0:
                n_neg=n_neg+1
            if row["Lag%s" % str(i+1)] > 0:
                n_pos=n_pos+1
        if n_pos>=lags*prec:
            res=1
        elif n_neg>=lags*prec:
            res=-1
        else:
            res=0
        sd.loc[z,'direction']=res
    prev_direction=0                # в след цикле проходим по значениям и предполагаем,  
    for z, row in sd.iterrows():    # что разворот - когда направление (direction) переходит из +1 в -1 или обратно.  
        if sd.loc[z,'direction']*prev_direction==-1:
            sd.loc[z,'change_direction']=1
        else:
            sd.loc[z,'change_direction']=0
        prev_direction=sd.loc[z,'direction']
          
def ShowMyFig(): 
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sd.index,y=sd["open"],mode='lines',name="open"))
    fig.add_trace(go.Scatter(x=sd.index,y=7350+50*sd["direction"],mode='lines',name="direction"))
    fig.add_trace(go.Scatter(x=sd.index,y=7200+100*sd["change_direction"],mode='lines',name="change_direction"))
    fig.show()

In [32]:
lags, prec =10, 0.9
SetDataFrameFromLagPrec(lags,prec)
ShowMyFig()
# график своеобразный и кривой, но вроде показательный. Наверняка можно сделать лучше. Но как? 

In [ ]:
# Параметры
# Как узнать разворот. 
# A Сравнить NN значений подряд, NN+1..NN+M значений подряд,  
# B Сравнивать N предыдущих значений с вероятностью P
ЧТО сравниваем? А. open.   

In [ ]:
N=4
P=0,9
sd["delta"]=sd["close"]-sd["open"]

In [ ]:
sd['open'].plot(fontsize=9)

In [ ]:
plt.plot(sd["open"])

In [ ]:
lags=3
for i in range(0, lags):
    sd["Lag%s" % str(i+1)] = sd["open"]-sd["open"].shift(i+1)
    sd["Vol%s" % str(i+1)] = sd["volume"].shift(i+1)
sd = sd.dropna()
sd

In [ ]:
# Заполняю столбец. Если курс вырос, то 1. Если нет, то '-1' 
sd["delta"]=sd["close"]-sd["open"]
sd["isgrowing"]=sd['delta'].map(lambda delta: math.copysign(1, delta ))

In [ ]:
# # Это просто построил график. Посмотреть на распределение   
# (sd['close']-sd['open']).hist(bins=10)

In [ ]:
train=sd[['Vol1','Vol2','Vol3','Lag1','Lag2','Lag3',]]
target=sd['isgrowing']
train_parm, test_parm, train_indic, test_indic = model_selection.train_test_split(
    train, target, test_size = 0.3, random_state = 0)

In [ ]:
model_rfc = RandomForestClassifier(n_estimators = 70) #в параметре передаем кол-во деревьев
model_knc = KNeighborsClassifier(n_neighbors = 18) #в параметре передаем кол-во соседей
model_lr = LogisticRegression(penalty='l1', tol=0.01) 
model_svc = svm.SVC()

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

kfold=5
itog_val = {}
scores = cross_val_score(model_rfc, train, target, cv = kfold)
itog_val['RandomForestClassifier'] = scores.mean()
scores = cross_val_score(model_knc, train, target, cv = kfold)
itog_val['KNeighborsClassifier'] = scores.mean()
scores = cross_val_score(model_lr, train, target, cv = kfold)
itog_val['LogisticRegression'] = scores.mean()
scores = cross_val_score(model_svc, train, target, cv = kfold)
itog_val['SVC'] = scores.mean()

In [ ]:
for i in itog_val: 
    print (itog_val[i])
DataFrame.from_dict(data = itog_val, orient='index').plot(kind='bar', legend=False)

In [ ]:
# УЖЕ ЛУЧШЕ. А ЧТО ЕСЛИ УБРАТЬ ВОСКРЕСЕНЬЕ. 
print ("Строк изначально",sd['open'].count())
sd = sd.query("WeekDayNmb  in [5]")
print ("Убрал день",sd['open'].count())
train=sd[['WeekDayNmb','empty']]
target=sd['isgrowing']
train_parm, test_parm, train_indic, test_indic = model_selection.train_test_split(
    train, target, test_size = 0.3, random_state = 0)

In [ ]:
model_rfc = RandomForestClassifier(n_estimators = 70) #в параметре передаем кол-во деревьев
model_knc = KNeighborsClassifier(n_neighbors = 18) #в параметре передаем кол-во соседей
model_lr = LogisticRegression(penalty='l1', tol=0.01) 
model_svc = svm.SVC()

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

kfold=5
itog_val = {}
scores = cross_val_score(model_rfc, train, target, cv = kfold)
itog_val['RandomForestClassifier'] = scores.mean()
scores = cross_val_score(model_knc, train, target, cv = kfold)
itog_val['KNeighborsClassifier'] = scores.mean()
scores = cross_val_score(model_lr, train, target, cv = kfold)
itog_val['LogisticRegression'] = scores.mean()
scores = cross_val_score(model_svc, train, target, cv = kfold)
itog_val['SVC'] = scores.mean()


In [ ]:
for i in itog_val: 
    print (itog_val[i])
DataFrame.from_dict(data = itog_val, orient='index').plot(kind='bar', legend=False)

In [ ]:
# Получается что в субботу с вероятностью 0,6 курс будет падать. Как-то так 

In [ ]:
# model_rfc.fit(train_parm, train_indic)
# model_lr.fit(train_parm, train_indic) 
# model_svc.fit(train_parm, train_indic)
# model_knc.fit(train_parm, train_indic)


# y_predicted = model_rfc.predict(test_parm)
# print(metrics.classification_report(test_indic, y_predicted))
# y_predicted = model_lr.predict(test_parm)
# y_predicted = model_svc.predict(test_parm)
# y_predicted = model_knc.predict(test_parm)


In [ ]:
# model_rfc.score(test_parm, test_indic)
# # print(metrics.classification_report(test_indic, y_predicted))
# # print(metrics.classification_report(test_indic, y_predicted))
# # print(metrics.classification_report(test_indic, y_predicted))
# # print(metrics.classification_report(test_indic, y_predicted))
